In [1]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr

In [2]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test, y_test = test.filter(regex=r'feature', axis=1), test['target_kazutsugi']

In [3]:
import glob
import re


preds_train1 = glob.glob("./preds_train1/*.pkl.z")
preds_val1 = glob.glob("./preds_val1/*.pkl.z")

df_train1 = []
for p_name in preds_train1:
    p = jb.load(p_name)
    p_name = re.search(r"train1/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_train1.append(p_df)
    
    
df_val1 = [] 
scores_val1 = dict()
for p_name in preds_val1:
    p = jb.load(p_name)
    p_name = re.search(r"val1/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_val1.append(p_df)
    scores_val1[p_name] = spearmanr(y_val1, p_df).correlation
    
    
df_train1 = pd.concat(df_train1, axis=1)
df_val1 = pd.concat(df_val1, axis=1)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [4]:
df_train1.shape

(178773, 208)

In [5]:
df_train1.columns[~df_train1.columns.isin(df_val1.columns)]

Index([], dtype='object')

In [6]:
df_val1.shape

(180938, 208)

In [7]:
df_test = [] 
scores_test = dict()
preds_test = glob.glob("./preds_test/*.pkl.z")
for p_name in preds_test:
    p = jb.load(p_name)
    p_name = re.search(r"test/(.*)\.pkl.z", p_name).group(1)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_test.append(p_df)
    scores_test[p_name] = spearmanr(y_test, p_df).correlation
    
    
df_test = pd.concat(df_test, axis=1)

In [30]:
selected = ['xgb_classifier', 'xgb_feature_groups_feature_charisma_feature_dexterity_feature_wisdom', 
            'xgb_feature_groups_feature_intelligence_feature_strength', 'lgbm_feature_groups_feature_constitution',
           'mlp_109_39_0.0009911332752617498', 'xgb_feature_groups_feature_intelligence_feature_charisma_feature_constitution',
           'lgbm_feature_groups_feature_strength_feature_dexterity', 'lgbm_feature_groups_feature_charisma_feature_dexterity_feature_constitution',
           'xgb_feature_groups_feature_intelligence_feature_wisdom']

In [32]:
from xgboost import XGBRegressor
best_score = dist.max() + 1e-4
#best_score = 0

for col in df_train1.columns:
    if col not in selected:
        Xtr = df_train1[selected+[col]].copy()
        Xval = df_val1[selected+[col]].copy()
    
    
        mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
        mdl.fit(Xtr, y_train1)

        p = mdl.predict(Xval)
        c = spearmanr(y_val1, p).correlation
        if c > best_score:
            print(col, c)
            best_score = c
    

# Permutation test

In [31]:
dist = []
for seed in range(20):
    Xtr = df_train1[selected].copy()
    Xval = df_val1[selected].copy()
    
    np.random.seed(seed)
    #print( Xtr.head())
    Xtr['random'] = np.random.permutation(Xtr.iloc[:, -1].values)
    Xval['random'] = np.random.permutation(Xval.iloc[:, -1].values)
    
    #print( Xtr.head())

    mdl = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0, colsample_bytree=0.1, subsample=0.25, 
                   tree_method='gpu_hist', gpu_id=0)
    mdl.fit(Xtr, y_train1)

    p = mdl.predict(Xval)
    c = spearmanr(y_val1, p).correlation
    #print(c)
    dist.append(c)
dist = np.array(dist)
dist.max()

0.04782067122801924

In [ ]:
# 0.04292171826668473
# 0.04458658639842637
# 0.045374108917674155
# 0.04590982547038226
# 0.04654385334527366
# 0.046885652294548705
# 0.04704532718564602
# 0.04751368628612044
# 0.04782067122801924